# **Importing Libraries and Tools**

In [1]:
!sudo pip install -U numpy==1.17.4.

Requirement already up-to-date: numpy==1.17.4. in /usr/local/lib/python3.6/dist-packages (1.17.4)


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
!apt-get install p7zip-full


In [0]:
import os
import numpy as np
import torch
from PIL import Image
import re

In [4]:
%%shell

# Install pycocotools
git clone https://github.com/cocodataset/cocoapi.git
cd cocoapi/PythonAPI
python setup.py build_ext install

fatal: destination path 'cocoapi' already exists and is not an empty directory.
running build_ext
skipping 'pycocotools/_mask.c' Cython extension (up-to-date)
running install
running bdist_egg
running egg_info
writing pycocotools.egg-info/PKG-INFO
writing dependency_links to pycocotools.egg-info/dependency_links.txt
writing requirements to pycocotools.egg-info/requires.txt
writing top-level names to pycocotools.egg-info/top_level.txt
writing manifest file 'pycocotools.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/pycocotools
copying build/lib.linux-x86_64-3.6/pycocotools/coco.py -> build/bdist.linux-x86_64/egg/pycocotools
copying build/lib.linux-x86_64-3.6/pycocotools/_mask.cpython-36m-x86_64-linux-gnu.so -> build/bdist.linux-x86_64/egg/pycocotools
copying build/lib.linux-x86_64-3.6/pycocotools/__init__.py -> build/bdist.linux-x86_64/egg/pycoc

In [5]:
%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.3.0

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

fatal: destination path 'vision' already exists and is not an empty directory.
HEAD is now at be37608 version check against PyTorch's CUDA version


In [0]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T

In [0]:
from torch.utils.data import Dataset
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


# **Getting Data**

In [0]:
from google.colab import drive
drive.mount('/gdrive')

In [0]:
cd /content/drive/My Drive/Projects/Vernacular Text Detection

/content/drive/My Drive/Projects/Vernacular Text Detection


In [0]:
!7z e real_Image_dataset_Detection.tar.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 46303949 bytes (45 MiB)

Extracting archive: real_Image_dataset_Detection.tar.7z
--
Path = real_Image_dataset_Detection.tar.7z
Type = 7z
Physical Size = 46303949
Headers Size = 166
Method = LZMA:24
Solid = -
Blocks = 1

  0%      2%      6% - real_Image_dataset_Detection.tar                                        10% - real_Image_dataset_Detection.tar                                        17% - real_Image_dataset_Detection.tar                                        23% 

In [0]:
!ls

Detection_data			     recognition_real_dataset.tar.7z
real_Image_dataset_Detection.tar     YOLO.ipynb
real_Image_dataset_Detection.tar.7z


In [0]:
!tar -xvf real_Image_dataset_Detection.tar

real_Image_dataset_Detection/
real_Image_dataset_Detection/readme.txt
real_Image_dataset_Detection/Image/
real_Image_dataset_Detection/Image/166.jpg
real_Image_dataset_Detection/Image/21.jpg
real_Image_dataset_Detection/Image/158. keralahouse_beefsaahil11.jpg
real_Image_dataset_Detection/Image/239.jpg
real_Image_dataset_Detection/Image/243.jpg
real_Image_dataset_Detection/Image/289.jpg
real_Image_dataset_Detection/Image/15.jpg
real_Image_dataset_Detection/Image/283.jpg
real_Image_dataset_Detection/Image/260.jpg
real_Image_dataset_Detection/Image/285.jpg
real_Image_dataset_Detection/Image/pathik-hotel-by-pass-choraha-indore-lodging-services-25wt7lv.jpg
real_Image_dataset_Detection/Image/281.jpg
real_Image_dataset_Detection/Image/210. 595291a7b4ce8a3750530dec_59267d9f9a22727e7f52d080_1498583463954.jpg
real_Image_dataset_Detection/Image/341. 15092450197_3dc5a760d3_b.jpg
real_Image_dataset_Detection/Image/314.jpg
real_Image_dataset_Detection/Image/72.jpg
real_Image_dataset_Detection/Image/

In [0]:
!ls

Detection_data			  real_Image_dataset_Detection.tar.7z
real_Image_dataset_Detection	  recognition_real_dataset.tar.7z
real_Image_dataset_Detection.tar  YOLO.ipynb


In [0]:
!mv real_Image_dataset_Detection	  detection_data

In [8]:
cd ~

/root


# **Creating Custom Dataset Class**

In [0]:
class DetectionDataset(Dataset):

  def __init__(self,root,transforms = None):

    self.root = root
    self.transforms = transforms

    self.images = list(sorted(os.listdir(os.path.join(root,'Image'))))
    self.masks = list(sorted(os.listdir(os.path.join(root,'Annotation'))))

  def __len__(self):
        return len(self.images)
        

  def __getitem__(self,index):

      img_path = os.path.join(self.root,'Image',self.images[index])
      mask_path = os.path.join(self.root,'Annotation',self.masks[index])

      image = Image.open(img_path)
      
      f = open(mask_path,'r')
      mask = f.readlines()
      f.close()


      coordinates_pattern = re.compile(r'(\d+)')
      label_pattern = re.compile(r'([A-Z]+)')

      boxes = []
      labels = []

      num_boxes = len(mask)

      for box in mask:

        coordinates = coordinates_pattern.findall(box)
        xmin = min(int(coordinates[0]),int(coordinates[2]))
        xmax = max(int(coordinates[0]),int(coordinates[2]))
        ymin = min(int(coordinates[4]),int(coordinates[6]))
        ymax = max(int(coordinates[4]),int(coordinates[6]))

        boxes.append([xmin, ymin, xmax, ymax])

        label = label_pattern.findall(box)
        labels.append(1 if label[0] == 'HINDI' else 0)
      
      boxes = torch.as_tensor(boxes, dtype=torch.float32)

      labels = torch.as_tensor(labels, dtype=torch.int64)

      iscrowd = torch.zeros(num_boxes, dtype=torch.int64)

      image_id = torch.tensor([index])

      area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

      target = {}

      target["boxes"] = boxes
      target["labels"] = labels
      target["image_id"] = image_id
      target["area"] = area
      target["iscrowd"] = iscrowd

      if self.transforms is not None:
          image, target = self.transforms(image, target)

      return image, target


In [0]:
data_path = '/content/drive/My Drive/Projects/Vernacular Text Detection/detection_data'

In [0]:
data = DetectionDataset(data_path)


In [12]:
data[20]

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=550x413 at 0x7FB0A5261A58>,
 {'area': tensor([1656., 1015., 1728., 2139.,  675., 1298.]),
  'boxes': tensor([[175., 191., 247., 214.],
          [250., 186., 285., 215.],
          [290., 189., 354., 216.],
          [174., 231., 267., 254.],
          [268., 230., 295., 255.],
          [294., 230., 353., 252.]]),
  'image_id': tensor([20]),
  'iscrowd': tensor([0, 0, 0, 0, 0, 0]),
  'labels': tensor([1, 1, 1, 0, 0, 0])})

# **Defining Helper Functions**

In [0]:
def get_detection_model(num_classes):

    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

    in_features = model.roi_heads.box_predictor.cls_score.in_features

    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

In [0]:
def get_transform(train):

    transforms = []
    transforms.append(T.ToTensor())

    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
        
    return T.Compose(transforms)

# **Loading Data**

In [0]:
dataset_train = DetectionDataset(data_path, get_transform(train=True))
dataset_test = DetectionDataset(data_path, get_transform(train=False))

In [0]:
torch.manual_seed(42)
indices = torch.randperm(len(dataset_train)).tolist()
dataset_train = torch.utils.data.Subset(dataset_train, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

In [0]:
data_loader_train = torch.utils.data.DataLoader(
    dataset_train, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=2, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

In [0]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# **Training and Evaluating Model**

In [19]:
num_classes = 3

model = get_detection_model(num_classes)
model.to(device)


FasterRCNN(
  (transform): GeneralizedRCNNTransform()
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d()
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d()
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d()
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d()
          (relu): ReLU(inplace=True)
          (downsample): Sequential(
            (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d()
          )
  

In [0]:
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [21]:
num_epochs = 10

for epoch in range(num_epochs):

    train_one_epoch(model, optimizer, data_loader_train, device, epoch, print_freq=10)

    lr_scheduler.step()
    
    evaluate(model, data_loader_test, device=device)

Epoch: [0]  [  0/189]  eta: 0:04:26  lr: 0.000032  loss: 1.2273 (1.2273)  loss_classifier: 1.1673 (1.1673)  loss_box_reg: 0.0450 (0.0450)  loss_objectness: 0.0075 (0.0075)  loss_rpn_box_reg: 0.0075 (0.0075)  time: 1.4126  data: 0.3073  max mem: 2225
Epoch: [0]  [ 10/189]  eta: 0:02:15  lr: 0.000297  loss: 1.9211 (2.7152)  loss_classifier: 0.9773 (0.9192)  loss_box_reg: 0.1294 (0.1666)  loss_objectness: 0.8017 (1.3384)  loss_rpn_box_reg: 0.0909 (0.2910)  time: 0.7579  data: 0.0343  max mem: 2828
Epoch: [0]  [ 20/189]  eta: 0:02:03  lr: 0.000563  loss: 1.2969 (1.8691)  loss_classifier: 0.4723 (0.6640)  loss_box_reg: 0.1779 (0.2053)  loss_objectness: 0.3560 (0.7892)  loss_rpn_box_reg: 0.0909 (0.2105)  time: 0.6958  data: 0.0070  max mem: 2828
Epoch: [0]  [ 30/189]  eta: 0:01:55  lr: 0.000829  loss: 0.8187 (1.5093)  loss_classifier: 0.3393 (0.5504)  loss_box_reg: 0.2296 (0.2201)  loss_objectness: 0.1270 (0.5776)  loss_rpn_box_reg: 0.0609 (0.1612)  time: 0.7094  data: 0.0067  max mem: 2845


# **Sample Inference**

In [0]:
img, _ = dataset_test[42]

model.eval()

with torch.no_grad():
    prediction = model([img.to(device)])

In [33]:
prediction

[{'boxes': tensor([[216.4278,  97.4728, 380.8792, 167.5460],
          [382.8749, 116.0938, 437.4785, 169.6785],
          [197.5161, 176.7950, 460.2957, 230.4183],
          [431.1786, 182.2305, 452.4231, 240.6882]], device='cuda:0'),
  'labels': tensor([1, 1, 1, 1], device='cuda:0'),
  'scores': tensor([0.9954, 0.6881, 0.2262, 0.0788], device='cuda:0')}]

In [34]:
dataset_test[42]

(tensor([[[0.8471, 0.8549, 0.8667,  ..., 0.4941, 0.4902, 0.4902],
          [0.8863, 0.8902, 0.8902,  ..., 0.4745, 0.4745, 0.4745],
          [0.8902, 0.8941, 0.8863,  ..., 0.4510, 0.4471, 0.4471],
          ...,
          [0.3098, 0.2118, 0.1412,  ..., 0.8196, 0.8157, 0.8078],
          [0.1882, 0.1765, 0.1804,  ..., 0.8118, 0.8039, 0.8000],
          [0.2039, 0.2157, 0.2157,  ..., 0.8196, 0.8118, 0.8078]],
 
         [[0.3725, 0.3804, 0.3922,  ..., 0.3176, 0.3216, 0.3216],
          [0.4118, 0.4157, 0.4235,  ..., 0.3059, 0.3059, 0.3059],
          [0.4157, 0.4196, 0.4196,  ..., 0.2902, 0.2941, 0.2941],
          ...,
          [0.2667, 0.1686, 0.0980,  ..., 0.5490, 0.5451, 0.5373],
          [0.1451, 0.1333, 0.1373,  ..., 0.5412, 0.5333, 0.5294],
          [0.1608, 0.1725, 0.1725,  ..., 0.5490, 0.5412, 0.5373]],
 
         [[0.3922, 0.4000, 0.4118,  ..., 0.2941, 0.2980, 0.2980],
          [0.4314, 0.4353, 0.4392,  ..., 0.2784, 0.2824, 0.2824],
          [0.4353, 0.4392, 0.4353,  ...,